# Import Library

In [ ]:
import numpy as np
import pandas as pd
import math
import random

# Data Label

In [ ]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

# Random Data

In [ ]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i+1)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

# Normalisasi

In [ ]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

# Ekstraksi File 


In [ ]:
def extraction(file, is_normal = True, is_random = True):
    data = np.array(pd.read_excel(file))
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data)
    return data

# K Cross Fold Validation

In [ ]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[9]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[9]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

# Euclidian Distance

In [ ]:
def euclidian(X, Y):
    euc = 0
    for i in range(len(X)):
        euc += math.pow((X[i]-Y[i]), 2)
    euc = math.sqrt(euc)
    return euc

# First Closest Centoid

In [ ]:
def closest(Xx, Xx_label, Y, cLabel):
    distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
    min_distance_value = [0 for i in range(cLabel)]
    min_distance_index = [0 for i in range(cLabel)]
    for i in range(len(Xx)):
        distance[((Xx_label[i][0])-1)][i] = euclidian(Xx[i], Y)
    for i in range(cLabel):
        min_distance_value[i] = min(x for x in distance[i] if x is not None)
        min_distance_index[i] = distance[i].index(min(x for x in distance[i] if x is not None))
    return np.array(distance), min_distance_index, min_distance_value

# Set of KNCN

In [ ]:
def kncn(k, Xx, Xx_label, Y, cLabel, close_dist):
    ncn = np.array([[0 for i in range(k)] for j in range(cLabel)])
    for i in range(cLabel):
        ncn[i][0] = close_dist[i]

    for i in range (1, k):
        distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
        min_distance_value = [0 for i in range(cLabel)]
        min_distance_index = [0 for i in range(cLabel)]
        for j in range(len(Xx)):
            label = Xx_label[j][0] - 1
            check = True
            for a in range(i):
                if ncn[label][a] == j:
                    check = False
            if check:
                add = 0 + Xx[j]
                for a in range(i):
                    indeks_ncn = ncn[label][a]
                    add += Xx[indeks_ncn]
                centroid = add / (i+1)
                distance[label][j] = euclidian(Y, centroid)
        for j in range(cLabel):
            min_distance_value[j] = min(x for x in distance[j] if x is not None)
            min_distance_index[j] = distance[j].index(min(x for x in distance[j] if x is not None))
            ncn[j][i] = min_distance_index[j]
    return ncn

# Local Mean Vector

In [ ]:
def localMeanVector(Xx, ncn):
    lm_ncn = [[0 for i in range(len(ncn[0]))] for j in range(len(ncn))]
    for i in range(len(ncn)):
        lm_ncn[i][0] = Xx[ncn[i][0]]
        for j in range(1, len(ncn[0])):
            add = 0 + lm_ncn[i][0]
            for a in range(j):
                add += Xx[ncn[i][a]]
            lm_ncn[i][j] = add / (j+1)
    return np.array(lm_ncn)

# Harmonic Mean Distance

In [ ]:
def harmonicMean(lm_ncn, Y):
    k = len(lm_ncn[0])
    hm_ncn = []
    for i in range(len(lm_ncn)):
        add = 0
        for j in range(len(lm_ncn[0])):
            distance = euclidian(Y, lm_ncn[i][j])
            ## Centroid Lokal i Tepat Pada Data Uji
            if distance == 0:
                add =+ 0
            else:
                add += (1/distance)
        ## Semua Centroid Lokal Tepat Pada Data Uji
        if add == 0:
            harmonic_mean = 0
        else:
            harmonic_mean = (k/add)
        hm_ncn.append(harmonic_mean)
    return hm_ncn

# Classification Decision

In [ ]:
def decision(hm_ncn, Y_label):
    min_distance_index = hm_ncn.index(min(x for x in hm_ncn))
    return (min_distance_index + 1), Y_label[0]

# LMKHNCN

In [ ]:
def lmkhncn(k, Xx, Xx_label, Yy, Yy_label, cLabel):
    acc = 0
    coff = np.array([[0 for i in range(jml_kelas)] for j in range(jml_kelas)])
    
    for i in range(len(Yy)):
        jarak, jarak_index_min, jarak_nilai_min = closest(Xx, Xx_label, Yy[i], cLabel)
        ncn = kncn(k, Xx, Xx_label, Yy[i], cLabel, jarak_index_min)
        local_mean = localMeanVector(Xx, ncn)
        harmonic_mean = harmonicMean(local_mean, Yy[i])
        predict, actual = decision(harmonic_mean, Yy_label[i])
        
        coff[predict-1][actual-1] += 1
        if(predict == actual):
            acc += 1
        print(str(round((i+1)/len(Yy),2)*100)+"%. Prediksi :"+ str(predict) + " Actual :" + str(actual) + " Acc Lokal: "+ str(acc/(i+1)))
    
    acc /= len(Yy)
    return coff

# Inisialisasi

In [ ]:
k = 1
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)
arr_kelas, jml_kelas = count_label(data)

# Testing

In [ ]:
for i in range(1, cross_val + 1):
    print("K Cross :"+str(i)+"---------------------------------------")
    data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
    coff = lmkhncn(k, train_f, train_l, test_f, test_l, jml_kelas)
    